# Basic jupyter notebook for shear bias computations

In [19]:
%run python/misc.py
%run python/simulations.py
%run python/shapes.py

In [2]:
#% run python/data.py
#download_HST_images()

In [3]:
# Job control:
job = misc.param()

# Re-run job if output file already exists?
job.re_run = False

# Set dry_run to True for test run
job.dry_run = False

In [4]:
# Small shear change for numerical derivative
dg = 0.02

In [5]:
# List of signs for shear change
# Four steps
g_steps = [(-1, 0), (0, -1), (1, 0), (0, 1)]

# Three steps, requires randomly changing signs
#g_step_list = [(-1, 0), (0, -1), (1, 1)]

In [6]:
# Dictionary and list of shears
g_dict = {}
for step in g_steps:
    g_dict[step] = (step[0] * dg, step[1] * dg)
g_values = g_dict.values()

### Setup

#### Image sizes and numbers

In [7]:
# Number of postage stamps is nxy_tiles^2
nxy_tiles = 2
#nxy_tiles = 100

# Number of files with different constant shear and PSF
nfiles    =  4
#nfiles   = 200

#### Galsim

In [8]:
# Files and directories
galsim_config_fname     = 'csc_multishear.yaml'
galsim_config_psf_fname = 'csc_psf.yaml'
galsim_config_dir       = 'config/galsim'
galsim_input_dir        = 'data/galsim/great3/control/space/constant'
galsim_output_base_dir  = 'output/galsim'

In [9]:
# Set paths
galsim_config_path     = '{}/{}'.format(galsim_config_dir, galsim_config_fname)
galsim_config_psf_path = '{}/{}'.format(galsim_config_dir, galsim_config_psf_fname)

### Create image simulations by calling galsim

In [ ]:
# TODO: Only run if output file does not exist
create_all_sims_great3(g_values, galsim_config_path, galsim_config_psf_path, galsim_input_dir, galsim_output_base_dir, nxy_tiles=nxy_tiles, nfiles=nfiles, job=job)

### Measure shapes

In [10]:
ksb_input_base_dir  = galsim_output_base_dir
ksb_output_base_dir = 'output/shapelens'
all_shapes_shapelens(g_values, ksb_input_base_dir, ksb_output_base_dir, nfiles, nxy_tiles, job=job)

*** Start all_shapes_shapelens ***
keeping existing file output/shapelens/g1_0.0_g2_0.02/result-000.txt, not running get_shapes -T -g 2 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.0_g2_0.02/image-000-0.fits > output/shapelens/g1_0.0_g2_0.02/result-000.txt
keeping existing file output/shapelens/g1_0.0_g2_-0.02/result-000.txt, not running get_shapes -T -g 2 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.0_g2_-0.02/image-000-0.fits > output/shapelens/g1_0.0_g2_-0.02/result-000.txt
keeping existing file output/shapelens/g1_0.02_g2_0.0/result-000.txt, not running get_shapes -T -g 2 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.02_g2_0.0/image-000-0.fits > output/shapelens/g1_0.02_g2_0.0/result-000.txt
keeping existing file output/shapelens/g1_-0.02_g2_0.0/result-000.txt, not running get_shapes -T -g 2 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_-0.02_g2_0.0/image-000-0.fits > output/shapelens/g1_-0.02_g2_0.

### Get shear estimate results

In [20]:
results_input_base_dir  = ksb_output_base_dir
psf_input_dir           = '{}/psf/'.format(galsim_output_base_dir)
results = all_read_shapelens(g_values, results_input_base_dir, psf_input_dir, nfiles)

*** Start all_read_shapelens v0.2 ***
File output/shapelens/g1_0.0_g2_0.02/result-000.txt, data dim (4, 5)
File output/shapelens/g1_0.0_g2_0.02/result-001.txt, data dim (4, 5)
File output/shapelens/g1_0.0_g2_0.02/result-002.txt, data dim (4, 5)
File output/shapelens/g1_0.0_g2_0.02/result-003.txt, data dim (4, 5)


NameError: global name 'step' is not defined

In [14]:
results[(-1,0)].e1

NameError: name 'results' is not defined

In [ ]:
R_output_dir = 'output/R'
R = shear_response(results, dg, output_dir=R_output_dir)

In [ ]:
len(results[(+1, 0)].e1)